In [ ]:
using CSV, Plots, Random, LinearAlgebra, Statistics, DataFrames

In [ ]:
num_of_model = 3
split = [43, 43+44, 43+44*2, 43+44*3, 43+44*4, 43+44*5]  # index for 5-fold

function select_best_model2(stock)  # using k-fold cross validation
    err = zeros(num_of_model,5)
    w = zeros(21)
    avg_trainw1 = []
    avg_testw1 = []
    avg_trainw2 = []
    avg_testw2 = []
    avg_trainw3 = []
    avg_testw3 = []
    
    ftw11 = zeros(6)
    ftw22 = zeros(7)
    ftw33 = zeros(8)
    
    ftw1 = zeros(6)
    ftw2 = zeros(7)
    ftw3 = zeros(8)
    
    for i in 1:5
        w_set = []
        # setting train and validation set
        split_cur = split[i]
        split2 = Int(round(split_cur*4/5))
        
        stock_train = stock[7][1:split2]
        stock_val = stock[7][split2 + 1:split_cur]
        
        sent_train = stock[1][1:split2]
        sent_val = stock[1][split2 + 1:split_cur]
        
        trav_train = stock[6][1:split2] #travelling
        trav_val = stock[6][split2 + 1:split_cur]
        
        death_train = stock[3][1:split2]
        death_val = stock[3][split2 + 1:split_cur]
        
        vac_train = stock[2][1:split2]
        vac_val = stock[2][split2 + 1:split_cur]
        
        shut_train = stock[4][1:split2]
        shut_val = stock[4][split2 + 1:split_cur]
    
        # Naive Model, w[1:6]
        Xtrain = [sent_train trav_train death_train vac_train shut_train ones(length(sent_train))]
        Xval = [sent_val trav_val death_val vac_val shut_val ones(length(sent_val))]
        Ytrain = stock_train
        Yval = stock_val
    
        w_cur = Xtrain\Ytrain
        w_set = vcat(w_set, w_cur)
        pred = Xval*w_cur
        err[1,i] = abs_err(Yval,pred)
        append!(avg_trainw1, w_cur)
        
        # AR1 Model, w[7:13] 1 to end - 1
        Xtrain = [sent_train[2:end] stock_train[1:end-1] trav_train[2:end] death_train[2:end] vac_train[2:end] shut_train[2:end] ones(length(sent_train)-1)]
        Xval = [sent_val[2:end] stock_val[1:end-1] trav_val[2:end] death_val[2:end] vac_val[2:end] shut_val[2:end] ones(length(sent_val)-1)]
        Ytrain = stock_train[2:end]
        Yval = stock_val[2:end]
    
        w_cur = Xtrain\Ytrain
        w_set = vcat(w_set, w_cur)
        pred = Xval*w_cur
        err[2,i] = abs_err(Yval,pred)
        append!(avg_trainw2, w_cur)
        
        # AR2 Model, w[14:21]
        Xtrain = [sent_train[3:end] stock_train[2:end-1] stock_train[1:end-2] trav_train[3:end] death_train[3:end] vac_train[3:end] shut_train[3:end] ones(length(sent_train)-2)]
        Xval = [sent_val[3:end] stock_val[2:end-1] stock_val[1:end-2] trav_val[3:end] death_val[3:end] vac_val[3:end] shut_val[3:end] ones(length(sent_val)-2)]
        Ytrain = stock_train[3:end]
        Yval = stock_val[3:end]
    
        w_cur = Xtrain\Ytrain
        w_set = vcat(w_set, w_cur)
        pred = Xval*w_cur
        err[3,i] = abs_err(Yval,pred)
        w = hcat(w,w_set)
        append!(avg_trainw3, w_cur)
    end
    
    avg_err = mean(err, dims=2)   # avg_abs_err  
    println("train error ", min(avg_err[2][1]))
    final_model = findmin(avg_err)[2][1]   # select model with min abs error
    #println("Low train error")
    
    if final_model == 1
        for i in 0:4
            ftw1[1] = ftw1[1] + avg_trainw1[1+i*6]
            ftw1[2] = ftw1[2] + avg_trainw1[2+i*6]
            ftw1[3] = ftw1[3] + avg_trainw1[3+i*6]
            ftw1[4] = ftw1[4] + avg_trainw1[4+i*6]
            ftw1[5] = ftw1[5] + avg_trainw1[5+i*6]
            ftw1[6] = ftw1[6] + avg_trainw1[6+i*6]
        end
        ftw1 = ftw1 ./ 5
        println("w train ", ftw1)
    elseif final_model == 2 
        for i in 0:4
            ftw2[1] = ftw2[1] + avg_trainw2[1+i*7]
            ftw2[2] = ftw2[2] + avg_trainw2[2+i*7]
            ftw2[3] = ftw2[3] + avg_trainw2[3+i*7]
            ftw2[4] = ftw2[4] + avg_trainw2[4+i*7]
            ftw2[5] = ftw2[5] + avg_trainw2[5+i*7]
            ftw2[6] = ftw2[6] + avg_trainw2[6+i*7]
            ftw2[7] = ftw2[7] + avg_trainw2[7+i*7]
        end
        ftw2 = ftw2 ./ 5
        println("w train ", ftw2)
    elseif final_model == 3
        for i in 0:4
            ftw3[1] = ftw3[1] + avg_trainw3[1+i*8]
            ftw3[2] = ftw3[2] + avg_trainw3[2+i*8]
            ftw3[3] = ftw3[3] + avg_trainw3[3+i*8]
            ftw3[4] = ftw3[4] + avg_trainw3[4+i*8]
            ftw3[5] = ftw3[5] + avg_trainw3[5+i*8]
            ftw3[6] = ftw3[6] + avg_trainw3[6+i*8]
            ftw3[7] = ftw3[7] + avg_trainw3[7+i*8]
            ftw3[8] = ftw3[8] + avg_trainw3[8+i*8]
        end
        ftw3 = ftw3 ./ 5
        println("w train ", ftw3)
    else
        print("ERROR")
    end
    
    test_err = zeros(5)
    model = ["Naive Model", "AR1 Model", "AR2 Model"]
    
    
    for i in 1:5
        stock_test = stock[7][split[i]+1:split[i+1]]
        sent_test = stock[1][split[i]+1:split[i+1]]
        trav_test = stock[6][split[i]+1:split[i+1]]
        death_test = stock[3][split[i]+1:split[i+1]]
        vac_test = stock[2][split[i]+1:split[i+1]]
        shut_test = stock[4][split[i]+1:split[i+1]]
    
        if final_model == 1   # Naive Model
            Xtest = [sent_test trav_test death_test vac_test shut_test ones(length(sent_test))]
            Ytest = stock_test
    
            w_cur = w[1:6, i+1]
            pred = Xtest*w_cur
            test_err[i] = abs_err(Ytest,pred)
            append!(avg_testw1,w_cur)
        
        elseif final_model == 2   # AR1 Model
            Xtest = [sent_test[2:end] stock_test[1:end-1] trav_test[2:end] death_test[2:end] vac_test[2:end] shut_test[2:end] ones(length(stock_test)-1)]
            Ytest = stock_test[2:end]
        
            w_cur = w[7:13, i+1]
            pred = Xtest*w_cur
            test_err[i] = abs_err(Ytest,pred)
            append!(avg_testw2,w_cur)
        
        elseif final_model == 3   # AR2 Model
            Xtest = [sent_test[3:end] stock_test[2:end-1] stock_test[1:end-2] trav_test[3:end] death_test[3:end] vac_test[3:end] shut_test[3:end] ones(length(stock_test)-2)]
            Ytest = stock_test[3:end]
        
            w_cur = w[14:21, i+1]
            pred = Xtest*w_cur
            test_err[i] = abs_err(Ytest,pred)
            append!(avg_testw3,w_cur)
        
        else
            print("error")
        end
    end
    if final_model == 1
        for i in 0:4
            ftw11[1] = ftw11[1] + avg_testw1[1+i*6]
            ftw11[2] = ftw11[2] + avg_testw1[2+i*6]
            ftw11[3] = ftw11[3] + avg_testw1[3+i*6]
            ftw11[4] = ftw11[4] + avg_testw1[4+i*6]
            ftw11[5] = ftw11[5] + avg_testw1[5+i*6]
            ftw11[6] = ftw11[6] + avg_testw1[6+i*6]
        end
        ftw11 = ftw11 ./ 5
        println("w test ", ftw1)
    elseif final_model == 2 
        for i in 0:4
            ftw22[1] = ftw22[1] + avg_testw2[1+i*7]
            ftw22[2] = ftw22[2] + avg_testw2[2+i*7]
            ftw22[3] = ftw22[3] + avg_testw2[3+i*7]
            ftw22[4] = ftw22[4] + avg_testw2[4+i*7]
            ftw22[5] = ftw22[5] + avg_testw2[5+i*7]
            ftw22[6] = ftw22[6] + avg_testw2[6+i*7]
            ftw22[7] = ftw22[7] + avg_testw2[7+i*7]
        end
        ftw22 = ftw22 ./ 5
        println("w test ", ftw2)
    elseif final_model == 3
        for i in 0:4
            ftw33[1] = ftw33[1] + avg_testw3[1+i*8]
            ftw33[2] = ftw33[2] + avg_testw3[2+i*8]
            ftw33[3] = ftw33[3] + avg_testw3[3+i*8]
            ftw33[4] = ftw33[4] + avg_testw3[4+i*8]
            ftw33[5] = ftw33[5] + avg_testw3[5+i*8]
            ftw33[6] = ftw33[6] + avg_testw3[6+i*8]
            ftw33[7] = ftw33[7] + avg_testw3[7+i*8]
            ftw33[8] = ftw33[8] + avg_testw3[8+i*8]
        end
        ftw33 = ftw33 ./ 5
        println("w test ", ftw3)
    else
        print("ERROR")
    end
    string("selected model: ", model[final_model], ", estimated error is: ", mean(test_err))
end